Mauri Leino

# Harjoitustyö vaihe 3 - Datan jalostaminen

__Pistetavoite: 2__

Datan keräämisvaiheessa päädyin lataamaan käyttööni yhteensä 5 csv-tiedostoa. Nyt tavoitteena on siivota datasetit, ja yhdistää ne yhdeksi datasetiksi, jota voin hyödyntää analysointivaiheessa. Tässä työvaiheessa yhdistelen pitkälti muiden linkkaamia esimerkkejä ja ohjeita, sillä vastaavanlaisia töitä on tehty aiemminkin.

## Airbnb-datan siistiminen

Airbnb-datasta on syytä muuttaa asuntojen hinnat numeroiksi, jotta niillä voidaan tehdä laskutoimituksia. Lisäksi on järkevää poistaa erikoisia arvoja sisältävät rivit. Tällaisia ovat esimerkiksi asunnot, joiden hinta on nolla, tai joihin on ilmoitettu mahtuvan nolla vierasta.

In [1]:
import pandas as pd
import numpy as np

cols = ['price',
        'neighbourhood_cleansed',
        'latitude',
        'longitude',
        'room_type',
        'accommodates',
        'bedrooms',
        'beds',
        'review_scores_rating',
        'review_scores_location',
        'number_of_reviews'
        ]

df = pd.read_csv('listings.csv', usecols=cols)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66644 entries, 0 to 66643
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   neighbourhood_cleansed  66641 non-null  object 
 1   latitude                66641 non-null  float64
 2   longitude               66641 non-null  float64
 3   room_type               66641 non-null  object 
 4   accommodates            66641 non-null  float64
 5   bedrooms                62624 non-null  float64
 6   beds                    62995 non-null  float64
 7   price                   66641 non-null  object 
 8   number_of_reviews       66641 non-null  float64
 9   review_scores_rating    48080 non-null  float64
 10  review_scores_location  46722 non-null  float64
dtypes: float64(8), object(3)
memory usage: 5.6+ MB


Muutetaan ensin asuntojen hinnat numeroiksi poistamalla dollarimerkit sekä mahdolliset tuhaterottimina käytettävät pilkut:

In [2]:
df['price'] = df['price'].str.strip('$')
df['price'] = df['price'].str.replace(",", "")
df['price'] = df['price'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66644 entries, 0 to 66643
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   neighbourhood_cleansed  66641 non-null  object 
 1   latitude                66641 non-null  float64
 2   longitude               66641 non-null  float64
 3   room_type               66641 non-null  object 
 4   accommodates            66641 non-null  float64
 5   bedrooms                62624 non-null  float64
 6   beds                    62995 non-null  float64
 7   price                   66641 non-null  float64
 8   number_of_reviews       66641 non-null  float64
 9   review_scores_rating    48080 non-null  float64
 10  review_scores_location  46722 non-null  float64
dtypes: float64(9), object(2)
memory usage: 5.6+ MB


In [3]:
# Katsotaan hintojen minimi ja maksimi
print(df.price.min())
print(df.price.max())

0.0
18557.0


Poistetaan ensinnäkin ne asunnot, joissa hinta on nolla, sekä lisäksi asunnot, joissa hinta/yö on yli 2000$, sillä ainakin osa näistä on todennäköisesti useamman asunnon komplekseja tai kohteita, joista ei ole listattu todenmukaista hintaa.

In [4]:
df = df[df.price != 0]
df = df[df.price <= 2000]
print(df.price.min())
print(df.price.max())
print(len(df.index))

7.0
2000.0
66447


Tässä emme menettäneet kauhean suurta määrää rivejä, noin 200 mikä ei alkuperäisestä ole kovin iso prosenttimäärä. Dataa on siis vielä tässä vaiheessa hyvin käytettävissä. Tarkastellaan seuraavaksi huoneistojen tyyppejä:

In [5]:
df['room_type'].value_counts()

Entire home/apt    37338
Private room       28363
Shared room          509
Hotel room           236
2.0                    1
Name: room_type, dtype: int64

Katsotaan mitä arvoja löytyy riviltä, jonka huonetyypiksi on merkitty 2.0:

In [6]:
print(df.loc[df['room_type'] == "2.0"])

      neighbourhood_cleansed  latitude  longitude room_type  accommodates  \
65878                    365       2.0      365.0       2.0         365.0   

       bedrooms  beds  price  number_of_reviews  review_scores_rating  \
65878       1.0  18.0   36.0                1.0                   NaN   

       review_scores_location  
65878                     NaN  


Rivi vaikuttaa hyvin erikoiselta, joten pudotetaan se datasetistä pois. Sen jälkeen korvataan puuttuvia arvoja nollalla ja poistetaan kohteet, joihin on ilmoitettu mahtuvan nolla vierasta, joissa on nolla vuodetta, tai joissa ei ole makuuhuoneita

In [7]:
df = df[df['room_type'] != "2.0"]

In [8]:
df['review_scores_rating'].fillna(0, inplace=True)
df['review_scores_location'].fillna(0, inplace=True)

df = df[df['accommodates'] != 0]
df = df[df['beds'] != 0]
df = df[df['bedrooms'] != 0]

df.info()
print(df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66446 entries, 0 to 66643
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   neighbourhood_cleansed  66446 non-null  object 
 1   latitude                66446 non-null  float64
 2   longitude               66446 non-null  float64
 3   room_type               66446 non-null  object 
 4   accommodates            66446 non-null  float64
 5   bedrooms                62509 non-null  float64
 6   beds                    62828 non-null  float64
 7   price                   66446 non-null  float64
 8   number_of_reviews       66446 non-null  float64
 9   review_scores_rating    66446 non-null  float64
 10  review_scores_location  66446 non-null  float64
dtypes: float64(9), object(2)
memory usage: 6.1+ MB
       neighbourhood_cleansed   latitude  longitude        room_type  \
0                   Islington  51.568610  -0.112700     Private room   
1     

Seuraavaksi tehdään datasetille kooditus kategorisoiduille kolumneille, joita ovat "neighbourhood_cleansed" sekä "room_type". Koodauksessa jokaista eri kolumnissa esiintyvää arvoa varten luodaan datasettiin uusi kolumni arvon nimellä, ja riveillä, joissa kukin arvo esiintyy, merkitään uuteen kolumniin lukuarvo 1 ja muihin uusiin kolumneihin 0. Näitä kolumneja tullaan käyttämään scikit-learn-kirjaston regressiossa.

In [9]:
df['neighbourhood_cleansed'] = df['neighbourhood_cleansed'].astype('category')
df['room_type'] = df['room_type'].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66446 entries, 0 to 66643
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   neighbourhood_cleansed  66446 non-null  category
 1   latitude                66446 non-null  float64 
 2   longitude               66446 non-null  float64 
 3   room_type               66446 non-null  category
 4   accommodates            66446 non-null  float64 
 5   bedrooms                62509 non-null  float64 
 6   beds                    62828 non-null  float64 
 7   price                   66446 non-null  float64 
 8   number_of_reviews       66446 non-null  float64 
 9   review_scores_rating    66446 non-null  float64 
 10  review_scores_location  66446 non-null  float64 
dtypes: category(2), float64(9)
memory usage: 5.2 MB


In [10]:
nh_dummies = pd.get_dummies(df['neighbourhood_cleansed'])
rt_dummies = pd.get_dummies(df['room_type'])

df = pd.concat((df, nh_dummies.astype(int), rt_dummies.astype(int)), axis=1)
columns = df.columns
print(len(columns))

48


Nyt käytössä on siis 48 kolumnia. Pudotetaan vielä kaikki loput tyhjiä arvoja sisältävät rivit datasetistä pois ja sen jälkeen ineksoidaan taulu uudelleen.

In [11]:
df= df.dropna()
df = df.reset_index(drop=True)

In [12]:
df

,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bedrooms,beds,price,number_of_reviews,review_scores_rating,...,Southwark,Sutton,Tower Hamlets,Waltham Forest,Wandsworth,Westminster,Entire home/apt,Hotel room,Private room,Shared room
0,Kensington and Chelsea,51.487800,-0.168130,Entire home/apt,2.0,1.0,1.0,75.0,89.0,4.79,...,0,0,0,0,0,0,1,0,0,0
1,Westminster,51.521950,-0.140940,Entire home/apt,6.0,3.0,3.0,265.0,43.0,4.69,...,0,0,0,0,0,1,1,0,0,0
2,Hammersmith and Fulham,51.479350,-0.197430,Private room,2.0,1.0,1.0,150.0,0.0,0.00,...,0,0,0,0,0,0,0,0,1,0
3,Hounslow,51.464160,-0.325540,Private room,2.0,1.0,1.0,65.0,11.0,4.55,...,0,0,0,0,0,0,0,0,1,0
4,Richmond upon Thames,51.480850,-0.280860,Entire home/apt,5.0,3.0,3.0,195.0,86.0,4.82,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59224,Croydon,51.419345,-0.079681,Entire home/apt,6.0,3.0,3.0,120.0,0.0,0.00,...,0,0,0,0,0,0,1,0,0,0
59225,Ealing,51.502002,-0.276895,Entire home/apt,8.0,3.0,4.0,288.0,0.0,0.00,...,0,0,0,0,0,0,1,0,0,0
59226,Haringey,51.599308,-0.114763,Entire home/apt,4.0,2.0,1.0,104.0,0.0,0.00,...,0,0,0,0,0,0,1,0,0,0
59227,Kensington and Chelsea,51.510407,-0.193729,Entire home/apt,4.0,2.0,2.0,402.0,0.0,0.00,...,0,0,0,0,0,0,1,0,0,0


Aribnb-datasetti vaikuttaa nyt siistiltä, joten tallennetaan se omaan tiedostoonsa myöhempää käyttöä varten.

In [13]:
df.to_csv('airbnb_data.csv')

## Loppujen datasettien yhdistäminen Aribnb:n dataan

Nyt kun Airbnb-data on siistitty, on aika siirtyä loppujen datasettien yhdistämiseen aiemmin tuotettuun datasettiin. Tähän sisältyy lähimmän kohteen etäisyyden laskeminen kullekin ulkoiselle datasetille jokaisen Airbnb-asunnon kohdalla asunnon ja kohteen koordinaattien avulla mpu-kirjaston avulla, ja lähimmän kohteen lisääminen Airbnb-datan sisältävään datasettiin. Määritellään ensin funktio, jolla voidaan laskea esimerkiksi etäisyys lähimpään pubiin annetuista koordinaateista.

In [14]:
# !pip install mpu
import mpu
import time

In [15]:
# lat, long: latitude and longitude of the point from which the distance to the closest attraction should be calculated
# file: name of file with information of attarctions
def shortest_distance(lat, long, file):
    cols = ['name', 'latitude', 'longitude']
    attraction_df = pd.read_csv(file, usecols=cols)
    first_found = False
    for i in range(len(attraction_df)):
        dist = mpu.haversine_distance((lat, long), (attraction_df['latitude'][i], attraction_df['longitude'][i]))
        if not first_found:
            first_found = True
            shortest = dist
        elif dist < shortest:
            shortest = dist
    return shortest

Kohteet sisältävät datasetit ovat tarvittavien tietojen osalta puhtaita jo valmiiksi ja kolumnit on nimetty kaikissa samalla tavalla, joten siivousta ei niiden kohdalla tarvitse tehdä. Tarkistetaan pienellä määrällä Airbnb-dataa, millaisia kilometrimääriä määritellyllä funktiolla saadaan.

In [16]:
for i in range(5):
    print(shortest_distance(df['latitude'][i], df['longitude'][i], 'cinemas.csv'))

0.1554625097969738
0.6327095717527662
0.31847763864901985
1.5268310589112564
1.1873888896378224


Etäisyydet vaikuttavat järkeviltä. Lasketaan seuraavaksi lähimmät etäisyydet kaikkiin erityppisiin kohteisiin kaikille Airbnb-asunnoille, ja lisätään tiedot datasettiin. Kuriositeettina päätin time-kirjaston avulla pyrkiä katsomaan myös, miten pitkään eri vaiheet prosessissa vievät.

In [17]:
init_start = time.perf_counter()
for file in ['cinemas', 'museums', 'musicvenues', 'pubs']:
    print("Calculating distance to " + file)
    # create empty array to collect values for each new column
    arr = []
    proc_start = time.perf_counter()
    for i in range(len(df)):
        #get shortest distance fom each Aribnb-apartment and append it to the array
        arr.append(shortest_distance(df['latitude'][i], df['longitude'][i], file + '.csv'))
    df[file] = arr
    proc_end = time.perf_counter()
    print(f"Calculated {file} in {proc_end - proc_start:0.4f} seconds")
init_end = time.perf_counter()
print(f"Calculated all targets in {init_end - init_start:0.4f} seconds")

Calculating distance to cinemas
Calculated cinemas in 290.2861 seconds
Calculating distance to museums
Calculated museums in 520.2128 seconds
Calculating distance to musicvenues
Calculated musicvenues in 1782.5546 seconds
Calculating distance to pubs
Calculated pubs in 7876.9525 seconds
Calculated all targets in 10470.0093 seconds


Kuten ajoista nähdään, prosessiin meni lopulta melko pitkään. Katsotaan vielä muutamaa riviä nyt tuotettujen kolumnien osalta, ja sen jälkeen tarkistetaan miltä kokonaisuus nyt näyttää.

In [21]:
distance_df = df[["cinemas", "museums", "musicvenues", "pubs"]]
distance_df.head()

,cinemas,museums,musicvenues,pubs
0,0.155463,0.464856,0.306629,0.039850
1,0.632710,0.204572,0.134450,0.120580
2,0.318478,1.527126,0.234239,0.057416
3,1.526831,1.404899,0.446330,0.143800
4,1.187389,0.745974,0.510942,0.363497


In [19]:
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59229 entries, 0 to 59228
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   neighbourhood_cleansed  59229 non-null  category
 1   latitude                59229 non-null  float64 
 2   longitude               59229 non-null  float64 
 3   room_type               59229 non-null  category
 4   accommodates            59229 non-null  float64 
 5   bedrooms                59229 non-null  float64 
 6   beds                    59229 non-null  float64 
 7   price                   59229 non-null  float64 
 8   number_of_reviews       59229 non-null  float64 
 9   review_scores_rating    59229 non-null  float64 
 10  review_scores_location  59229 non-null  float64 
 11  Barking and Dagenham    59229 non-null  int64   
 12  Barnet                  59229 non-null  int64   
 13  Bexley                  59229 non-null  int64   
 14  Brent                 

,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bedrooms,beds,price,number_of_reviews,review_scores_rating,...,Wandsworth,Westminster,Entire home/apt,Hotel room,Private room,Shared room,cinemas,museums,musicvenues,pubs
0,Kensington and Chelsea,51.487800,-0.168130,Entire home/apt,2.0,1.0,1.0,75.0,89.0,4.79,...,0,0,1,0,0,0,0.155463,0.464856,0.306629,0.039850
1,Westminster,51.521950,-0.140940,Entire home/apt,6.0,3.0,3.0,265.0,43.0,4.69,...,0,1,1,0,0,0,0.632710,0.204572,0.134450,0.120580
2,Hammersmith and Fulham,51.479350,-0.197430,Private room,2.0,1.0,1.0,150.0,0.0,0.00,...,0,0,0,0,1,0,0.318478,1.527126,0.234239,0.057416
3,Hounslow,51.464160,-0.325540,Private room,2.0,1.0,1.0,65.0,11.0,4.55,...,0,0,0,0,1,0,1.526831,1.404899,0.446330,0.143800
4,Richmond upon Thames,51.480850,-0.280860,Entire home/apt,5.0,3.0,3.0,195.0,86.0,4.82,...,0,0,1,0,0,0,1.187389,0.745974,0.510942,0.363497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59224,Croydon,51.419345,-0.079681,Entire home/apt,6.0,3.0,3.0,120.0,0.0,0.00,...,0,0,1,0,0,0,2.129378,0.316706,0.034544,0.027403
59225,Ealing,51.502002,-0.276895,Entire home/apt,8.0,3.0,4.0,288.0,0.0,0.00,...,0,0,1,0,0,0,2.303189,0.636393,0.698589,0.523311
59226,Haringey,51.599308,-0.114763,Entire home/apt,4.0,2.0,1.0,104.0,0.0,0.00,...,0,0,1,0,0,0,0.624466,2.826844,1.143130,0.287419
59227,Kensington and Chelsea,51.510407,-0.193729,Entire home/apt,4.0,2.0,2.0,402.0,0.0,0.00,...,0,0,1,0,0,0,0.276931,0.811413,0.201449,0.177604


Muodostettu datasetti näyttää hyvältä, joten tallennetaan se vielä omaan tiedostoonsa ja näin datan jalostusvaihe on saatu valmiiksi.

In [20]:
df.to_csv('data.csv')

## Mietteitä ja linkkejä

<ul>
    <li>Tämä vaihe oli aiemmin harjoitustyön tehneiden vinkkaamien esimerkkien pohjalta melko yksinkertainen, ja ainoa varsinainen haaste oli soveltaa nuo tiedot omaan työhön sopiviksi. Siis se, mitä kolumneja esimerkiksi valitaan, tai miten omaa dataa on tarpeen siivota. Lisäksi laskennan tekeminen vei paljon aikaa, ja tähän kannattanee varautua. Hyvä pohja Airbnb-datan siivoamiselle on kurssin esimerkki: <a href="https://github.com/InfoTUNI/joda2022/blob/master/koodiesimerkit/airnbn/python_scikit_airbnb.ipynb" target="_blank">https://github.com/InfoTUNI/joda2022/blob/master/koodiesimerkit/airnbn/python_scikit_airbnb.ipynb</a>
    <li>GCP temppuili muutamaan otteeseen sulkien Jupyter Labin ja toisinaan yhteyden koko virtuaalikoneeseen, kun otin yhteyden Googlen Cloud Shellin kautta. Pääsin kuitenkin ongelmista ongelmista eroon ainakin tähän asti, kun lopetin Cloud Shellin käytön ja otin SSH-yhteyden tietokoneeni komentoriviltä.
    <li>mpu-kirjasto oli todella kätevä etäisyyksien laskemiseen koordinaattien avulla. mpu-dokumentaatio: <a href="https://mpu.readthedocs.io/en/latest/mpu.html" target="_blank">https://mpu.readthedocs.io/en/latest/mpu.html</a>
    <li>Pandasin avulla dataframejen muokkaaminen on todella helppoa, ja Pythonilla omien funktioiden määrittely käy nopeasti. Pandasin dokumentaatio: <a href="https://pandas.pydata.org/docs/user_guide/index.html" target="_blank">https://pandas.pydata.org/docs/user_guide/index.html</a>
</ul>
